## Solving bidomain equations on rectangular domain with the Beeler-Reuter model - literature

### Importing required libraries

In [1]:
%cd ..
from src.dynamics_models import *
from src.cell_models import *
from src.utils import *

/home/drvinko/Desktop/Modeliranje-biofizike-srcanog-misica


### Defining a domain, cell model and dynamics model and visualizing initial data

In [2]:
domain = rectangle(1, 1, 59, 59)

In [3]:
class CellModel(BeelerReuter):
    def applied_current(self, t: float):
        t_0 = 5
        tau_0 = 1.5
        r_0 = 0.125

        def value(x):
            r = np.sqrt((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2) / r_0
            tau = (t - t_0) / tau_0
            if 0 < tau < 1:
                return (
                    50
                    * (1 + np.cos(np.pi * r))
                    / 2
                    * (1 + np.cos(np.pi * tau))
                    / 2
                    * (r < 1)
                )
            else:
                return np.zeros_like(x[0])

        self.I_app.interpolate(value)


class Model(MonodomainModel):
    def initial_V_m(self):
        self.V_m_n.x.array[:] = -82

    def conductivity(self):
        fibers = ufl.as_vector([1, 0])
        M_il = 1.741e-3
        M_el = 3.906e-3
        M_it = 0.1934e-3
        M_et = 1.970e-3
        self.M_i = M_it * ufl.Identity(2) + (M_il - M_it) * ufl.outer(fibers, fibers)
        self.M_i = 2*self.M_i

In [4]:
cell_model = CellModel(domain)
model = Model(domain, cell_model)

### Solving equations with given parameters

In [5]:
model.solve(
    T=16,
    steps=2000,
    lambda_=1,
    save_to="test_BR.mp4",
    camera_direction="xy",
    zoom=0.9,
    checkpoints=[6,8,10,12,14,16],
    checkpoint_file="test_BR",
)

Solving problem: 100%|██████████| 2000/2000 [19:32<00:00,  1.71it/s]
